Adaptado de:
https://github.com/piegu/language-models/blob/master/HuggingFace_Notebook_token_classification_NER_LeNER_Br.ipynb

# 1. Configs


In [1]:
#teste gpu
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Feb 16 14:15:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#teste ram
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
#drive e variáveis globais
#utilizar GPU
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
#installs
%%capture
!pip install datasets transformers seqeval
!apt install git-lfs

In [7]:
#imports
import pathlib
from pathlib import Path
import pandas as pd
import transformers
import datasets
from datasets import Dataset, DatasetDict
from datasets import load_dataset, load_metric

print(transformers.__version__) 
print(datasets.__version__) 

4.16.2
1.18.3


In [8]:
#BERT pre-treinado Lener
model_checkpoint = "pierreguillou/bert-base-cased-pt-lenerbr"
#dtaset para fine-tunning
datasets = load_dataset("lener_br")
#tarefa
task = "ner" 

#entidades no modelo IOB
#datasets["train"].features[f"ner_tags"]
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/70.8k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset lener_br downloaded and prepared to /root/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

['O',
 'B-ORGANIZACAO',
 'I-ORGANIZACAO',
 'B-PESSOA',
 'I-PESSOA',
 'B-TEMPO',
 'I-TEMPO',
 'B-LOCAL',
 'I-LOCAL',
 'B-LEGISLACAO',
 'I-LEGISLACAO',
 'B-JURISPRUDENCIA',
 'I-JURISPRUDENCIA']

In [ ]:
#mostrar elementos do dataset
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

show_random_elements(datasets["train"])

# 2. Pre-processing

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
label_all_tokens = True

'''
example = datasets["train"][5]
print(example["tokens"])
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))
label_all_tokens = True
'''

'\nexample = datasets["train"][5]\nprint(example["tokens"])\ntokenized_input = tokenizer(example["tokens"], is_split_into_words=True)\ntokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])\nprint(tokens)\nword_ids = tokenized_input.word_ids()\naligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]\nprint(len(aligned_labels), len(tokenized_input["input_ids"]))\nlabel_all_tokens = True\n'

In [18]:
#tokenização
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenize_and_align_labels(datasets['train'][:5])
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382/cache-0580e70418ba42b9.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382/cache-bc0a4474270e3cd0.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382/cache-c7a163454afa8ec1.arrow


# 3. Finetunning

In [19]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading:   0%|          | 0.00/893 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at pierreguillou/bert-base-cased-pt-lenerbr were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at pierreguillou/be

# 4. Treinamento

## 4.1 Connfigurações do modelo

In [21]:

model_name = model_checkpoint.split("/")[-1]

# hyperparameters utilizado no teste
per_device_batch_size = 10  #[5, 10, 20] cada batch por thread, duplicando os valores.
num_train_epochs = 55 # [35, 55, 75] pierre: we started with 10 epochs but the model overfits fastly
learning_rate = 2e-5 #[1e-4,2e-5,3e-4,5e-5] pierre: (AdamW) we started with 3e-4, then 1e-4, then 5e-5 but the model overfits fastly

# hyperparameters não modificados
gradient_accumulation_steps = 2
weight_decay = 0.01
save_total_limit = 100 ##early stopping
logging_steps = 290 #melhor evaluate frequently (5000 seems too high)
eval_steps = logging_steps
evaluation_strategy = 'epoch' #pierre; 'steps'
logging_strategy = 'epoch' #'pierre; 'steps'
save_strategy = 'epoch' #pierre; 'steps'
save_steps = logging_steps
load_best_model_at_end = True

fp16 = True

# folders
model_name = model_checkpoint.split("/")[-1]
folder_model = 'e' + str(num_train_epochs) + '_lr' + str(learning_rate)
output_dir = '/content/drive/MyDrive/Colab Notebooks/projeto_DL/projetoDL_BERT/output/' + 'ner-lenerbr-' + str(model_name) + '/checkpoints/' + folder_model
logging_dir = '/content/drive/MyDrive/Colab Notebooks/projeto_DL/projetoDL_BERT/logs/' + 'ner-lenerbr-' + str(model_name) + '/logs/' + folder_model

# metrics
metric_for_best_model = 'eval_f1'
if metric_for_best_model == 'eval_f1':
    greater_is_better = True
elif metric_for_best_model == 'eval_loss':
    greater_is_better = False  

args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_batch_size, 
    per_device_eval_batch_size=per_device_batch_size*2,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_train_epochs, 
    weight_decay=weight_decay,
    save_total_limit=save_total_limit,
    logging_steps = logging_steps,
    eval_steps = logging_steps,
    load_best_model_at_end = load_best_model_at_end,
    metric_for_best_model = metric_for_best_model,
    greater_is_better = greater_is_better,
    gradient_checkpointing = False,
    do_train = True,
    do_eval = True,
    do_predict = True,
    evaluation_strategy = evaluation_strategy,
    logging_dir=logging_dir, 
    logging_strategy = logging_strategy,
    save_strategy = save_strategy,
    save_steps = save_steps,
    fp16 = fp16,
    push_to_hub=False,
)

In [24]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")
#labels = [label_list[i] for i in example[f"{task}_tags"]]
#metric.compute(predictions=[labels], references=[labels])

In [25]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## 4.2 Execução

In [ ]:
from transformers.trainer_callback import EarlyStoppingCallback

# wait early_stopping_patience x eval_steps before to stp the training in order to get a better model
early_stopping_patience = save_total_limit

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
)

trainer.train()

# 5. Avaliação

In [27]:
#geral
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens.
***** Running Evaluation *****
  Num examples = 1177
  Batch size = 20


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,0.217200,0.127338,0.807813,0.831613,0.819540,0.961677
1,0.047200,0.094852,0.834428,0.901720,0.866770,0.973319
1,0.047200,0.094852,0.834428,0.901720,0.866770,0.973319


{'eval_accuracy': 0.9733189083761257,
 'eval_f1': 0.8667700258397932,
 'eval_loss': 0.09485199302434921,
 'eval_precision': 0.8344278606965174,
 'eval_recall': 0.9017204301075269}

In [28]:
#predição por entidades
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens.
***** Running Prediction *****
  Num examples = 1177
  Batch size = 20


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,0.217200,0.127338,0.807813,0.831613,0.819540,0.961677
1,0.047200,0.094852,0.834428,0.901720,0.866770,0.973319
1,0.047200,0.094852,0.834428,0.901720,0.866770,0.973319


{'JURISPRUDENCIA': {'f1': 0.6703755215577191,
  'number': 657,
  'precision': 0.617157490396927,
  'recall': 0.7336377473363774},
 'LEGISLACAO': {'f1': 0.8468013468013469,
  'number': 571,
  'precision': 0.8152350081037277,
  'recall': 0.8809106830122592},
 'LOCAL': {'f1': 0.759493670886076,
  'number': 194,
  'precision': 0.746268656716418,
  'recall': 0.7731958762886598},
 'ORGANIZACAO': {'f1': 0.8480392156862745,
  'number': 1340,
  'precision': 0.7988126649076517,
  'recall': 0.9037313432835821},
 'PESSOA': {'f1': 0.9868421052631579,
  'number': 1072,
  'precision': 0.9943181818181818,
  'recall': 0.9794776119402985},
 'TEMPO': {'f1': 0.9544910179640718,
  'number': 816,
  'precision': 0.9332552693208431,
  'recall': 0.9767156862745098},
 'overall_accuracy': 0.9733189083761257,
 'overall_f1': 0.8667700258397932,
 'overall_precision': 0.8344278606965174,
 'overall_recall': 0.9017204301075269}